In [1]:
# Whole Project Folder has to be given to students
# Student have to install anaconda 
# Students also need the envrionemnt file 
# They activate the environment they cd to Project folder 
# THEN RUN "jupyter notebook"
# Browser windows opens 
# And they can run the cells 


In [2]:
# training the model - train.py

In [3]:
from __future__ import print_function

import argparse
import time
import os
from six.moves import cPickle

In [4]:
class args:
    data_dir = 'data/space'
    save_dir = 'save'
    log_dir = 'logs'
    save_every = 1000
    init_from = None
    model = 'lstm'
    rnn_size = 128
    num_layers = 2 
    seq_length = 50 
    batch_size = 50 
    num_epochs = 2 
    grad_clip = 5.
    learning_rate = 0.002 
    decay_rate = 0.97 
    output_keep_prob = 1.0 
    input_keep_prob = 1.0 

In [5]:
import tensorflow as tf
from utils import TextLoader
from model import Model

In [6]:
def train(args):
    data_loader = TextLoader(args.data_dir, args.batch_size, args.seq_length)
    args.vocab_size = data_loader.vocab_size
    
    # check compatibility if training is continued from previously saved model
    if args.init_from is not None:
        # check if all necessary files exist
        assert os.path.isdir(args.init_from)," %s must be a a path" % args.init_from
        assert os.path.isfile(os.path.join(args.init_from,"config.pkl")),"config.pkl file does not exist in path %s"%args.init_from
        assert os.path.isfile(os.path.join(args.init_from,"chars_vocab.pkl")),"chars_vocab.pkl.pkl file does not exist in path %s" % args.init_from
        ckpt = tf.train.latest_checkpoint(args.init_from)
        assert ckpt, "No checkpoint found"

        # open old config and check if models are compatible
        with open(os.path.join(args.init_from, 'config.pkl'), 'rb') as f:
            saved_model_args = cPickle.load(f)
        need_be_same = ["model", "rnn_size", "num_layers", "seq_length"]
        for checkme in need_be_same:
            assert vars(saved_model_args)[checkme]==vars(args)[checkme],"Command line argument and saved model disagree on '%s' "%checkme

        # open saved vocab/dict and check if vocabs/dicts are compatible
        with open(os.path.join(args.init_from, 'chars_vocab.pkl'), 'rb') as f:
            saved_chars, saved_vocab = cPickle.load(f)
        assert saved_chars==data_loader.chars, "Data and loaded model disagree on character set!"
        assert saved_vocab==data_loader.vocab, "Data and loaded model disagree on dictionary mappings!"

    if not os.path.isdir(args.save_dir):
        os.makedirs(args.save_dir)
    with open(os.path.join(args.save_dir, 'config.pkl'), 'wb') as f:
        cPickle.dump(args, f)
    with open(os.path.join(args.save_dir, 'chars_vocab.pkl'), 'wb') as f:
        cPickle.dump((data_loader.chars, data_loader.vocab), f)

    model = Model(args)

    with tf.Session() as sess:
        # instrument for tensorboard
        summaries = tf.summary.merge_all()
        writer = tf.summary.FileWriter(
                os.path.join(args.log_dir, time.strftime("%Y-%m-%d-%H-%M-%S")))
        writer.add_graph(sess.graph)

        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver(tf.global_variables())
        # restore model
        if args.init_from is not None:
            saver.restore(sess, ckpt)
        for e in range(args.num_epochs):
            sess.run(tf.assign(model.lr,
                               args.learning_rate * (args.decay_rate ** e)))
            data_loader.reset_batch_pointer()
            state = sess.run(model.initial_state)
            for b in range(data_loader.num_batches):
                start = time.time()
                x, y = data_loader.next_batch()
                feed = {model.input_data: x, model.targets: y}
                for i, (c, h) in enumerate(model.initial_state):
                    feed[c] = state[i].c
                    feed[h] = state[i].h

                # instrument for tensorboard
                summ, train_loss, state, _ = sess.run([summaries, model.cost, model.final_state, model.train_op], feed)
                writer.add_summary(summ, e * data_loader.num_batches + b)

                end = time.time()
                print("{}/{} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}"
                      .format(e * data_loader.num_batches + b,
                              args.num_epochs * data_loader.num_batches,
                              e, train_loss, end - start))
                if (e * data_loader.num_batches + b) % args.save_every == 0\
                        or (e == args.num_epochs-1 and
                            b == data_loader.num_batches-1):
                    # save for the last result
                    checkpoint_path = os.path.join(args.save_dir, 'model.ckpt')
                    saver.save(sess, checkpoint_path,
                               global_step=e * data_loader.num_batches + b)
                    print("model saved to {}".format(checkpoint_path))


if __name__ == '__main__':
    train(args)

loading preprocessed files
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



0/298 (epoch 0), train_loss = 4.439, time/batch = 0.337
model saved to save\model.ckpt
1/298 (epoch 0), train_loss = 4.404, time/batch = 0.029
2/298 (epoch 0), train_loss = 4.326, time/batch = 0.028
3/298 (epoch 0), train_loss = 4.062, time/batch = 0.028
4/298 (epoch 0), train_loss = 3.638, time/batc

91/298 (epoch 0), train_loss = 2.682, time/batch = 0.029
92/298 (epoch 0), train_loss = 2.661, time/batch = 0.028
93/298 (epoch 0), train_loss = 2.737, time/batch = 0.028
94/298 (epoch 0), train_loss = 2.700, time/batch = 0.028
95/298 (epoch 0), train_loss = 2.680, time/batch = 0.028
96/298 (epoch 0), train_loss = 2.637, time/batch = 0.029
97/298 (epoch 0), train_loss = 2.663, time/batch = 0.028
98/298 (epoch 0), train_loss = 2.625, time/batch = 0.028
99/298 (epoch 0), train_loss = 2.721, time/batch = 0.028
100/298 (epoch 0), train_loss = 2.585, time/batch = 0.028
101/298 (epoch 0), train_loss = 2.552, time/batch = 0.028
102/298 (epoch 0), train_loss = 2.612, time/batch = 0.028
103/298 (epoch 0), train_loss = 2.536, time/batch = 0.028
104/298 (epoch 0), train_loss = 2.550, time/batch = 0.028
105/298 (epoch 0), train_loss = 2.531, time/batch = 0.028
106/298 (epoch 0), train_loss = 2.533, time/batch = 0.028
107/298 (epoch 0), train_loss = 2.531, time/batch = 0.028
108/298 (epoch 0), trai

239/298 (epoch 1), train_loss = 2.306, time/batch = 0.028
240/298 (epoch 1), train_loss = 2.306, time/batch = 0.028
241/298 (epoch 1), train_loss = 2.295, time/batch = 0.028
242/298 (epoch 1), train_loss = 2.347, time/batch = 0.029
243/298 (epoch 1), train_loss = 2.342, time/batch = 0.029
244/298 (epoch 1), train_loss = 2.309, time/batch = 0.028
245/298 (epoch 1), train_loss = 2.283, time/batch = 0.028
246/298 (epoch 1), train_loss = 2.320, time/batch = 0.028
247/298 (epoch 1), train_loss = 2.285, time/batch = 0.028
248/298 (epoch 1), train_loss = 2.381, time/batch = 0.028
249/298 (epoch 1), train_loss = 2.247, time/batch = 0.028
250/298 (epoch 1), train_loss = 2.230, time/batch = 0.028
251/298 (epoch 1), train_loss = 2.283, time/batch = 0.028
252/298 (epoch 1), train_loss = 2.223, time/batch = 0.028
253/298 (epoch 1), train_loss = 2.246, time/batch = 0.028
254/298 (epoch 1), train_loss = 2.239, time/batch = 0.028
255/298 (epoch 1), train_loss = 2.243, time/batch = 0.029
256/298 (epoch